# Introduction & Background

https://mkhosla-ucsd.github.io/cogs138/final-group-project/part-one

### Overview

 ❏    Write a clear summary of what you did

 ❏    Limit overview to 3-4 sentences


### Research Question
 ❏    Include your _specific_ data science question

 ❏    Make sure what you’re measuring (variables) to answer your question is clear


### Background & Prior Work
 ❏    Include explanation of what work has been done previously

 ❏    Include citations or links to previous work

 ❏    Include an explanation of dataset(s) used (_i.e._, how the data was collected, features/variables included, number of observations, information in dataset)


### Hypothesis
 ❏    Include your team’s hypothesis

 ❏    Ensure that this hypothesis is clear to readers

 ❏    Explain why you think this will be the outcome (what was your thinking?)


# Data Analysis

In [6]:
# import essentials
import scipy
import pandas as pd
import numpy as np

# methods & metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.datasets import make_blobs

# util
import PIL
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import warnings
import os
from util import *

### Wrangling
 ❏    Pull your data into Python and explain steps taken to isolate the data you need

In [8]:
non_demented = dataloader('data/Non_Demented')
mild_demented = dataloader('data/Mild_Demented')
moderate_demented = dataloader('data/Moderate_Demented')
very_mild_demented = dataloader('data/Very_Mild_Demented')

Loaded 3200 images with shape (128, 128)
Loaded 896 images with shape (128, 128)
Loaded 64 images with shape (128, 128)
Loaded 2240 images with shape (128, 128)


### Visualization
 ❏    Include at least three visualizations

 ❏    Clearly label all axes on plots

 ❏    Type of all plots appropriate given data displayed

 ❏    Interpretation of each visualization included in text


### Results
 ❏    Appropriate analysis performed

 ❏    Output of analysis interpreted and interpretation included in notebook


# Conclusion & Discussion
❏    Discussion of your results and how they address your experimental question

❏    Limitations of analysis discussed

❏    What additional experiments would be interesting, and what data would you need?